<a href="https://colab.research.google.com/github/suyeon-9706/MNIST/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PyTorch 
- Scientific Computing Package based on Python (Deep Learning Library)
- Dynamic Computing Graph (DCG) Support
- Easy access to python arrangement features
- Excellent compatibility with numpy / scipy.
- Most are implemented in python except C++ code for tensor operation.

# MNIST
- Provided on Yann LeCun's website
- A simple computer vision data set
- It consists of handwritten images.
- 28*28 image, 1 channel gray image, 0~9 digits
- Each data is labeled 'What is the number of that data' with the data.






In [0]:
#code: https://github.com/AvivSham/Pytorch-MNIST-colab/blob/master/Pytorch_MNIST.ipynb
#@title Install pytorch
!pip install torch
!pip install torchvision

In [0]:
import torch
import torch.nn as nn # Packages for creating a Neural Network
# torch.autograd: A package that is central to the Neural network, providing automatic differentiation for all operations of the Tensor

from torch.autograd import Variable # Variable class: core class of autograd package

# torchvision: use for image classification training, easy to vision training
import torchvision.datasets as dsets # Data loader for datasets such as CIFAR10, MNIST, etc.
import torchvision.transforms as transf # A package that transforms images of PIL type into torch tensor type

In [0]:
import matplotlib.pyplot as plt # visualization package(Used to draw a graph)
import numpy as np # For matrix operations
import random #For random operations

In [3]:
#@title Loading MNIST data
mnist_train = dsets.MNIST(root='data/',
                          train=True, # train set
                          transform=transf.ToTensor(), # image to Tensor 
                          download=True) # If MNIST image does not exist in root, download data
mnist_test = dsets.MNIST(root='data/',
                         train=False, # val(test set)
                         transform=transf.ToTensor(), # image to Tensor
                         download=True) # If MNIST image does not exist in root, download data

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 23317177.51it/s]                            


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 452005.17it/s]
  1%|          | 16384/1648877 [00:00<00:11, 142143.33it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7416169.70it/s]                           
8192it [00:00, 173741.12it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
#@title Create batch operator to enter data in batch units
# To update the parameters of a model: gradient descent(Update parameters after reporting batch size of data)
batch_size = 100
train_data = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size, 
                                         shuffle=True) # shuffle data
test_data = torch.utils.data.DataLoader(dataset=mnist_test, 
                                        batch_size=batch_size, 
                                        shuffle=False) # don't shuffle data

*batch_size*: the size of input data took for one iteration

In [5]:
#@title Define model
print("Define model...")
class Net(nn.Module):
  # Initialize all modules here(instantiate)
  def __init__(self, input_size, hidden_size, num_classes):    
    super(Net, self).__init__() # Always 'torch.nn.Module' inheritance, then start
    self.fc1 = nn.Linear(input_size, hidden_size) # used for linear transformation
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes) # used for linear transformation
    
  # Define the network structure
  # A function in which the model receives training data and proceeds to 'forward propagation'
  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

Define model...


*input_size*: img_size(MNIST data image of shape 28*28 = 784)

*hidden_size*: number of nodes at hidden layer

*num_classes*: number of output classes(MNIST label: discrete range [0,9])

*Linear() function*: Creates an 'input_size' image into a one-dimensional vector => Outputs 'num_classes' classes via the 'hidden_size' nodes at hidden layer

*RELU() function*: a function treated as zero only for negative numbers, such as max(0, x)


---

The reason they are grouped together into a module is because of the 'gpu allocation'.

( graph를 선언할 때 gpu option을 주면 
그 안에서 선언한 함수는 모두 gpu에 할당되는 Tensorflow와는 다르게 
PyTorch에서는 직접 .cuda() 를 통해 할당시켜주어야 한다. 
이 번거로움을 최소화시키고자, 모듈로 구성.)

In [0]:
#@title Build the model
input_size = 784 # img_size = (28,28) -> 28*28=784 in total
hidden_size = 500
num_classes = 10 # discrete range [0,9]

net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  net.cuda()

*torch.cuda.is_available() function*: Returns a bool indicating if CUDA is currently available. (Verify that GPUs are available in given environment)

*cuda()*: Used to replace the existing Tensor with a data type that allows GPU operation


In [0]:
#@title Define loss & optimizer
# loss
loss_function = nn.CrossEntropyLoss()
# backpropagation method
learning_rate = 1e-3
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

*nn.CrossEntropyLoss() function*: calculate 'cross entropy loss'
<br><br>
*torch.optim*: a package implementing various optimization algorithms. 

*torch.optim.Adam(params, lr=~)*:  An algorithm for first-order "gradient-based optimization" of stochastic objective functions, based on adaptive estimates of lower-order moments.

- *params(iterable)*: iterable of parameters to optimize or dicts defining parameter groups

- *lr(float, otional)*: learning rate(default: le-3) (1e-3 --> 1∗10^−3=0.001)

- straightforward to implement(1), computationally efficient(2), little memory requirements(3), well suited for problems that are large in terms of data and/or parameters(4)
